<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<p>
<h1>Algorísmica Avançada</h1>
<h2>Pràctica 1 - Grafs </h2>
</center>
</p>
</div>

<div class="alert alert-danger" style="width:95%; margin:0 auto; padding">
<center><p><h2> ¡¡IMPORTANTE!! </h2></p> </center> 

<p>
Para la realizacón de esta práctica tendréis que utilizar vuestra propia clase `Graph` implementada en la Práctica 0. Si dicha clase no cumple los requisitos funcionales y no pasa los tests, con muy alta probabilidad los tests propuestos para esta práctica tampoco funcionen correctamente. 
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Introducción</h1>
</center>


A lo largo de esta práctica trabajaremos con el grafo generado a partir de la red de metro de Londres. En este grafo los nodos representan las estaciones y los ejes las vias que van de una estación a otra. Todos los ejes tienen cuatro atributos:

* Linea
* Color
* Nombre (de la linea)
* Distancia


In [ ]:
# TODO: Importar grafo de la Práctica0
from util import get_subway_graph, draw_subway_graph
from graph import Graph

G, lines = get_subway_graph('csv', Graph)
G.edge[1][52]

Los nodos contienen el nombre de la estación, la latitud y longitud a la que está situada la estación, el número de lineas y la zona.

In [ ]:
G.node[124]

Para mas consultas, la información ha sido extraida de Wikimedia Commons:

https://commons.wikimedia.org/wiki/London_Underground_geographic_maps/CSV

# util.py

En este archivo se os facilitan tres funciones que os permitiran cargar y visualizar la red de metro.
```python
"""
Retorna un objeto nx.Graph que corresponde al grafo de la red de metro y un 
diccionario con las lineas del metro
 - location: ruta donde esta almacenado el archivo .csv
 - Klass: la clase Graph que hemos desarrollado en la práctica 0
"""
G, lines = get_subway_graph(location, Klass)

"""
Dibuja el grafo que le pasemos por parametro.
- G: Grafo de la red de metro
- lines: diccionario con la información sobre las lineas del metro
- figsize: parametro opcional que nos permite definir el tamaño de la figura
- show_labels: parametro opcional que nos permite indicar si queremos mostrar los 
    nombres de las estaciones
"""
draw_metro_graph(G, lines, figsize=(10,6), show_labels=False)

```

In [ ]:
draw_subway_graph(G, lines, figsize=(10,6))

<div class="alert alert-info">
<center>
  <h1>Contenido</h1>
  </center><p>



<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>1- Dijkstra</p></h2>
  
  <p>
 Se propone que hagáis dos implementaciones del algoritmo <a href="https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm">Dijkstra</a>. 
 <ul><li>
 **dijkstra1:** La primera consiste en el algoritmo común en el que tendremos en cuenta las distancias definidas en los edges. 
 </li><li>
 **dijkstra2:** La segunda implementación es simplemente una ampliación del primer algoritmo en el que añadiremos una penalización por cambio de linea.</li>
 </ul>
</p>
<h3> Ejemplo _dijkstra2_</h3>
<p>
Para ilustrar lo que se pide en la práctica veamos como se calcularía el valor de **C** y **D** en el siguiente gráfico:
</p>


<p><center><img src='img/e1.png'></img></center></p>


<p>
<ul><li>
**[C]** Calculamos V(B) como V(A) + d([A,B]), calculamos V(C) como V(B) + d([B,C]), ya hemos acabado.
</li>
<li>
**[D]** Calculamos V(B) como V(A) + d([A,B]), calculamos V(D) como V(B) + d([B,D]), como [A,B] es linea azul y [B,D] es linea verde, añadimos una penalización por el transbordo a la que llamamos **P**. Por lo tanto, al final obtenemos que V(D) = V(B) + d([B,C]) + P
</li>
</ul>

<br>
-**V(X)**: Valor de X
<br>
-**d([X,Y])**: Distancia de X a Y
<br>
-**P**: Penalización por transbordo

</p>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center>
<p>
<h3>INPUT</h3>
<ul>
<li>__G__: Este es el grafo (en el caso de esta práctica la red de metro) que utilizaremos para buscar el camino. Debe de ser un objeto de tipo `Graph` como el que habéis implementado en la Práctica 0.</li>
<li>__origen__: Este parámetro corresponde al índice de un nodo. En este caso, como indexamos los nodos con el identificador de las paradas de Metro, deberá ser un entero _(e.g. 231)_.</li>
<li>__destino__: El indice del nodo al que queremos llegar.</li>
<li>__infinity=*(int)*__: Parametro opcional en el que definimos que numero nos va bien para utilizar como infinito en el momento de inicializar los pesos de los nodos.</li>
<li>__penalty=*(int)*__: (_Solo necesario para dijkstra2)_ Es un numero entero que corresponde a la penalización que aplicamos al valor de un nodo cuando hay cambio de linea.
</ul>
<br>
<h3>OUTPUT</h3>
El output de la función es un diccionario que contiene los siguientes valores:
<ul>
<li>__ _'path'_ __: Una lista de índices correspondientes al camino encontrado del nodo inicial al nodo final __ambos nodos, inicio y final, han de estar incluidos en esta lista__.</li>
<li>__ _'expanded'_ __: El numero de nodos que se han visitado para encontrar la solución.</li>
<li>__ _'distance'_ __: La distancia del camino mínimo desde el origen hasta el destino (es decir, el valor del nodo destino).
<ul>

</p>
</div>


In [1]:
import heapq

def dijkstra(G, origen, destino, infinity=float('inf')):
    
    path = [(0, origen)]
    expanded = 0
    
    for iteratorNode in G.nodes():
        G.node[iteratorNode].update({'distance': infinity, 
                          'previous': None, 
                          'visited' : False})
        
    G.node[origen]['distance'] = 0
    
    while (len(path) != 0):
        actualNode = heapq.heappop(path)[1]
        
        if (actualNode == destino):
            break
            
        if not (G.node[actualNode]['visited']):
            expanded += 1
            G.node[actualNode]['visited'] = True
            
            for neighborNode in G.neighbors(actualNode):
                actualNode_and_edge_distance = G.node[actualNode]['distance'] + G.edge[actualNode][neighborNode]['distance']
                
                if (G.node[neighborNode]['distance'] > actualNode_and_edge_distance):
                    G.node[neighborNode]['distance'] = actualNode_and_edge_distance
                    G.node[neighborNode]['previous'] = actualNode
                    
                    heapq.heappush(path, (G._nodes[neighborNode]['distance'], neighborNode))

    path = [destino]
    
    while not (G.node[path[-1]]['previous'] == None):
        path.append(G.node[path[-1]]['previous'])
        
    path.reverse()
    
    return {'path': path, 
            'expanded': expanded, 
            'distance': G.node[destino]['distance']}


In [2]:
dijkstra(G, 10, 235)

NameError: name 'G' is not defined

In [ ]:
import heapq

def dijkstra2(G, origen, destino, penalty = 5000000, infinity = float('inf')):

    path = [(0, origen)]
    expanded = 0
    
    for iteratorNode in G.nodes():
        G.node[iteratorNode].update({'distance': infinity, 
                          'previous': None, 
                          'visited' : False})
        
    # DIJKSTRA 2    
    
    G.node[origen]['distance'] = 0
    
    while (len(path) != 0):
        actualNode = heapq.heappop(path)[1]
        
        if (actualNode == destino):
            break
            
        if not (G.node[actualNode]['visited']):
            expanded += 1
            G.node[actualNode]['visited'] = True
            
            for neighborNode in G.neighbors(actualNode):
                actualNode_and_edge_distance = G.node[actualNode]['distance'] + G.edge[actualNode][neighborNode]['distance']
                
                if (G.node[neighborNode]['distance'] > actualNode_and_edge_distance):
                    G.node[neighborNode]['distance'] = actualNode_and_edge_distance
                    G.node[neighborNode]['previous'] = actualNode
                    
                    heapq.heappush(path, (G._nodes[neighborNode]['distance'], neighborNode))

    path = [destino]
    
    while not (G.node[path[-1]]['previous'] == None):
        path.append(G.node[path[-1]]['previous'])
        
    path.reverse()
    
    return {'path': path, 
            'expanded': expanded, 
            'distance': G.node[destino]['distance']}

In [ ]:
dijkstra2(G, 10, 235)

<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios Dijkstra</h3></p> </center> </div>

<b>Dijkstra1</b>

Primero de todo, he creado tres nuevos atributos a los nodos del grafo, siendo estos <i>distance</i>, el cual almacena la distancia acumulada desde el nodo origen hasta el nodo actual, <i>previous</i>, el cual se usa para almacenar el nodo previo al nodo acutal en el recorrido de Dijkstra por el grafo, y, por último, <i>visited</i>, que indica si ese nodo ya ha sido visitado durante el recorrido o no lo ha estado.

He asignado los siguientes valores a los tres atributos antes mencionados:

<ul style="list-style-type:disc">
  <li><i>distance</i>: infinity (ya que todos los nodos tienen distancia infinita por no ser vecinos del nodo origen).</li>
  <li><i>previous</i>: None (ya que no pueden haber nodos previos en el camino si aún no hay ningún camino definido).</li>
  <li><i>visited</i>: False (ya que aún no se ha visitado ningún nodo).</li>
</ul>  

Después de esto, he asignado el atributo <i>distance</i> del nodo origen como 0, ya que ahí empieza el recorrido.

Habiendo hecho esto, puedo empezar a iterar.

Para cada iteración, se actualiza el nodo (el mínimo de toda la lista) a partir del cual trabajaremos. Habiendo seleccionado el nodo, comprobamos si ese es el nodo destino, si eso se cumpliese se acabaría la búsqueda, aunque si no fuese así, la búsqueda continuaría. Ahora miramos si el nodo actual ha sido ya visitado, si ha sido visitado actualizamos el nodo actual, y si no es así, guardamos en el path el nodo vecino y su distancia si la distancia del vecino es mayor a la distancia del nodo actual más la de la arista que los une. Además de eso, también se guarda el nodo previo del recorrido para cada nodo actual.

Cuando este bucle termina (cuando no quedan más elementos en el path, ya que a cada iteración se elimina un elemento) se recorre el path del final al principio encadenando los nodos a través de su nodo previo, de tal manera que queda el recorrido al revés, únicamente quedando invertir la lista y retornar los resultados.



La complejidad de este algoritmo es <b>(E + V) * log V</b>, donde E son las aristas y V son los vértices.

<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios Dijkstra2</h3></p> </center> </div>

### _(En esta sección se os propone explicar como habeis realizado la implementación y cual es la complejidad detallada del algoritmo. Podéis contestar en este mismo bloque)_

<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>2- Matriz de distancias</p></h2>
   <p>
 Se propone el desarrollo de un algoritmo que dado un grafo construya una matriz de $V \times V$ (donde $V$ es el número de vértices del grafo) que contenga la distancia mínima de cada vértice al resto. 
</p>

<p></p>

<p>
<b> Nota: </b> Recordad que en Algorítmica Avanzada buscamos la implementación de algoritmos que no solo resuelvan el problema, sino que lo hagan de manera eficiente
</p>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center>
<p>
<h3>INPUT</h3>
<ul>
<li>__G__: Este es el grafo (en el caso de esta práctica la red de metro) sobre el que calcularemos la matriz de distancias. Debe de ser un objeto de tipo `Graph` como el que habéis implementado en la Práctica 0.</li>
</ul>
<br>
<h3>OUTPUT</h3>
<ul>
<li>El output de la función es un diccionario cuya clave es un nodo (origen) y el valor es otro diccionario con la forma $\{nodo\_destino: distancia\}$.</li>
    </ul>

</p>
</div>


In [ ]:
import heapq

def dijkstraModificat(G, origen, infinity=float('inf')):
    
    path = [(0, origen)]
    distancias = {}
    
    for iteratorNode in G.nodes():
        G.node[iteratorNode].update({'distance': infinity, 
                                     'visited' : False})
        
    G.node[origen]['distance'] = 0
    
    while (len(path) != 0):
        actualNode = heapq.heappop(path)[1]
                    
        if not (G.node[actualNode]['visited']):
            G.node[actualNode]['visited'] = True
            
            for neighborNode in G.neighbors(actualNode):
                actualNode_and_edge_distance = G.node[actualNode]['distance'] + G.edge[actualNode][neighborNode]['distance']
                
                if (G.node[neighborNode]['distance'] > actualNode_and_edge_distance):
                    G.node[neighborNode]['distance'] = actualNode_and_edge_distance
                    
                    heapq.heappush(path, (G._nodes[neighborNode]['distance'], neighborNode))
    
            for iteratorNode in G.nodes():
                distancias[iteratorNode] = G._nodes[iteratorNode]['distance']
    
    return distancias

In [ ]:
def dist_mat(G):
    
    matrix = {}
    
    for iteratorNode in G.nodes():
        matrix[iteratorNode] = dijkstraModificat(G, iteratorNode)
            
    return matrix

In [ ]:
dist_mat(G)

<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios Matriz de Distancias</h3></p> </center> </div>

<b>Matriz de distancias</b>

Para hacer este ejercicio he tenido que modificar el algoritmo de Dijkstra1 de tal manera que únicamente reciba como parámetro el nodo origen y éste devuelva la distancia desde el nodo origen a todos los demás nodos del grafo, de tal manera que, en el ejercicio de la matriz de distancias, únicamente será necesario iterar sobre nodos del grafo y llamar a la función de Dijkstra modificada para obtener las distancias y guardarlo todo en un diccionario.

La complejidad de este algoritmo es <b>O(n)</b>.

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>
<p>
La entrega de esta práctica se podrá realizar en el campus virtual hasta el día __21 de Octubre a las 23:55__. En la tarea que se habilitará en el campus deberéis colgar __el archivo .ipynb__ asi como __el fichero .pyc con vuestra clase grafo__ con los nombres:
</p>
<p>
```
* AA_P1_<apellido1>_<apellido2>_<nombre>.ipynb
* graphs.pyc
```

</p>
<p>
Por ejemplo:
</p>
<p>
```
AA_P1_Doe_Ritchie_John.ipynb
```

</p>
<p>

Es fundamental que el código esté bién comentado y con un análisis de complejidad exhaustivo del algoritmo.
</p>
</div>